# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [8]:
pip install jupytergmaps

ERROR: Could not find a version that satisfies the requirement jupytergmaps (from versions: none)
ERROR: No matching distribution found for jupytergmaps

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('../output_data/cities.csv')
df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
...,...,...,...,...,...,...,...,...,...,...
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564
574,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565
575,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,1666108566


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Accesss the map with a unique API key
gmaps.configure(api_key=g_key)

# Use Lat and Lng as locations and Humidity as the weight
locations = df[["Lat", "Lng"]]
weights = df["Humidity"]

# Customize the size of the figure and add a heatmap layer to the map
figure_layout = {
    'width': "100%",
    'height': '400%',
    'margin': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)

# Set maximum intensity and point_radius settings
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

fig

Figure(layout=FigureLayout(height='400%', margin='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Create a dataframe with conditional statements to fit the below criteria 
# A max temperature lower than 50 degrees but higher than 20
# Wind Speed less than 15 mph
# Cloudiness at least 50%
# Humidity less than 30%

ideal_city = df.loc[(df["Max Temp"] > 70) &
                    (df["Max Temp"] < 20) &
                    (df["Wind Speed"] < 15) &
                    (df["Cloudiness"] > 70) &
                    (df["Humidity"] < 70), :].reset_index(drop=True).dropna()

ideal_city

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store into variable names hotel_df

hotel_df = ideal_city[["City", "Country", "Lat", "Lng"]].copy()

# Add a "Hotel Name" column to the DataFrame with null values
hotel_df["Hotel Name"] = np.nan
hotel_df

In [ ]:
# Set up parameters for Google Places API
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# Set up Dictionary
parameters = {
    "keyword" : target_search,
    "radius" : target_radius,
    "type" : target_type,
    "key" : g_key
}

# Define base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:
# Use a for loop to go through each city in the DataFrame and make API call

for index, row in hotel_df.iterrows():
    
    # Get the coordinates and store in parameters dictionary
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"] = f"{lat},{lng}"
    
    # Call API and get hotel details for each city
    hotel_data = requests.get(base_url, parameters).json()
    
    # Use exception handling to save the first hotel name for each city
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("---------------------------------------------------------------")
        
    except:
        print(f"Could not find information for city '{row['City']}'... Skipping.")
        print("---------------------------------------------------------------")

In [ ]:
# Remove null rows from DataFrame
hotel_df.dropna(how="any", inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# Display final DataFrame
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """

Name
{Hotel Name}

City
{City}
Country
{Country}

"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]